#### Imports

In [1]:
import PyPDF2
import csv
import re
import json
import os
import us

#### Scrape key to facility codes
- Very inclusive info about keys, many things not keys included

In [2]:
year = 2003
pdfFileObj = open('{0}/directory_{0}.pdf'.format(year), 'rb')
pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
print("Number of pages:", pdfReader.numPages)

Number of pages: 742


In [50]:
def clean_key_page(page, offset):
    raw_abbr_key = pdfReader.getPage(page - 1).extractText()
    m = re.findall(r'[A-Z]{3}', raw_abbr_key)[offset:]
    return [l[:2] for l in m]

In [51]:
abbr_keys = ['MH-SA']
abbr_keys += clean_key_page(5, 4)

In [52]:
for i in range(1, 68):
    abbr_keys.append('N' + str(i))

In [53]:
for i in range(1, 114):
    abbr_keys.append('F' + str(i))

In [54]:
abbr_keys

['MH-SA',
 'SA',
 'MH',
 'SA',
 'GH',
 'TX',
 'DT',
 'MM',
 'LA',
 'DM',
 'LA',
 'HH',
 'OP',
 'PH',
 'RR',
 'HI',
 'AD',
 'CO',
 'HV',
 'HI',
 'AI',
 'GL',
 'SE',
 'PW',
 'WO',
 'MN',
 'DU',
 'DW',
 'CJ',
 'MD',
 'MC',
 'PI',
 'MI',
 'TR',
 'CA',
 'SF',
 'SS',
 'PA',
 'AH',
 'SP',
 'N1',
 'N2',
 'N3',
 'N4',
 'N5',
 'N6',
 'N7',
 'N8',
 'N9',
 'N10',
 'N11',
 'N12',
 'N13',
 'N14',
 'N15',
 'N16',
 'N17',
 'N18',
 'N19',
 'N20',
 'N21',
 'N22',
 'N23',
 'N24',
 'N25',
 'N26',
 'N27',
 'N28',
 'N29',
 'N30',
 'N31',
 'N32',
 'N33',
 'N34',
 'N35',
 'N36',
 'N37',
 'N38',
 'N39',
 'N40',
 'N41',
 'N42',
 'N43',
 'N44',
 'N45',
 'N46',
 'N47',
 'N48',
 'N49',
 'N50',
 'N51',
 'N52',
 'N53',
 'N54',
 'N55',
 'N56',
 'N57',
 'N58',
 'N59',
 'N60',
 'N61',
 'N62',
 'N63',
 'N64',
 'N65',
 'N66',
 'N67',
 'F1',
 'F2',
 'F3',
 'F4',
 'F5',
 'F6',
 'F7',
 'F8',
 'F9',
 'F10',
 'F11',
 'F12',
 'F13',
 'F14',
 'F15',
 'F16',
 'F17',
 'F18',
 'F19',
 'F20',
 'F21',
 'F22',
 'F23',
 'F24',
 'F25',

#### Scraper test on a page of data
- data starts on page 15 (getPage(14))

In [289]:
def split_data(rawdata):
    '''
    Roughly split data, returns list of list.
    '''
    #split erroneously connected data
    data = []
    temp = []
    i = 0
    while i < len(rawdata) - 1:
        #if i is the last line of keys, i+1 is facility name not phone number/address
        if (rawdata[i].split()[0] in abbr_key) and (not rawdata[i+1].split()[0] in abbr_key):
        #and (not rawdata[i+1][-4:].isnumeric()): 
            temp.append(rawdata[i])
            data.append(temp)
            temp = []
        else:
            temp.append(rawdata[i])
        i += 1
    temp.append(rawdata[i])
    data.append(temp)
    return data

In [290]:
def split_center_street(cn_sa):
    '''
    Split a string containing center name and street address into two.
    Inputs:
        cn_sa(str)
    Returns: a list
    '''
    #Exception to split at number :
        #When it ends with "P.O. Box",
        #When the number being found is at the very front (wrongly included page number or 
        #center name starts with number)
        
    #If the number is at the end(suite xxx or building xxx) lets include the word in front of it)
    #print(cn_sa)
    #if cn_sa[0].isnumeric():
    #    cn_sa[0].replace(' For Code Definitions - See KEY on Page v. and cover flap', '')

    if cn_sa[0].isupper() and cn_sa[1].isnumeric():
        m = re.search(r"\d+", cn_sa).end()
        cn_sa = cn_sa[m:]
        
    m = re.finditer(r"\d+", cn_sa)

    for num in m:
        span = num.span()
        if span[0] != 0:    
            if cn_sa[span[0] - 9:span[0] - 1] == 'P.O. Box':
                return ([cn_sa[:span[0] - 10], cn_sa[span[0] - 9:]])
            if span[1] != len(cn_sa):
                return ([cn_sa[:span[0] - 1], cn_sa[span[0]:]])
            if span[1] == len(cn_sa):
                return ([' '.join(cn_sa.split()[:-2]), ' '.join(cn_sa.split()[-2:])])
    return [cn_sa, '']

In [291]:
def clean_data(writer, cts, year, pagen):
    '''
    Move and link wrongly splitted data, changes in place
    '''
    #deal with first center
    data_holders = []
    data_holder = [year, pagen]
    ct = cts[0]
    #print(ct)
    cn_sn_all = ''
    i = 0
    while i < len(ct):
        if len(ct[i]) > 5 and ',' in ct[i] and ct[i][-5:].isnumeric():
            break
        cn_sn_all += ' ' + ct[i]
        i += 1
    cn_sn_all = cn_sn_all.strip()
    #print(cn_sn_all)

    m = re.search(r'[A-Z ]{5}[A-Z ]+', cn_sn_all)
    if m == None:
        m = 1
    else:
        m = m.end()
    data_holder += split_center_street(cn_sn_all[m-1:])
    #City, State, Postal_code
    city, state_postcode = ct[i].split(',')
    state, postcode = state_postcode.split()
    data_holder += [city, state, postcode]
    i += 1
    #phone + keys
    phone = ct[i]
    i += 1
    while not ct[i][0:2].isupper():
        phone += ' ' + ct[i]
        i += 1
    if phone[-1].isupper():
        key_start = re.search(r'[A-Z]{2}', phone).start()
        data_holder.append(phone[:key_start])
        data_holder.append(phone[key_start:] + ' ' + ' '.join(ct[i:]))
    else:
        key_start = len(phone)
        data_holder.append(phone)
        data_holder.append(' '.join(ct[i:]))
    #Clean the keys in last line also split the other center name if present
    keys_tail = data_holder[-1]
    if not ' '.join(keys_tail).isupper():
        keys = keys_tail.split(' ')
        data_holder[-1] = ''
        i = 0
        while i < len(keys):
            if not keys[i].isupper():
                break
            i += 1
        data_holder[-1] = ' '.join(keys[:i])
        morekey_cn = ' '.join(keys[i:])
        i = 0
        while i < len(morekey_cn):
            if morekey_cn[:i] in abbr_keys:
                if morekey_cn[:i+1] in abbr_keys: #like F46
                    i += 1
                break
            i += 1
        if i == len(morekey_cn):
            i = 0
        data_holder[-1] += morekey_cn[:i]
        cts[1] = [morekey_cn[i:]] + cts[1]
    #write data
    #writer.writerow(data_holder)
    data_holders.append(data_holder)
    
    
    #rest of it
    j = 1
    while j < len(cts):
        ct = cts[j]
        #print(ct, '\n') 
        if len(ct) < 3: 
            data_holders[-1][-1] += ' '.join(ct)
        else:
            data_holder = [year, pagen]
            i = 0
            while ((len(ct[i]) < 5) or (',' not in ct[i]) or not (ct[i][-5:].isnumeric())): 
                i += 1
            stuff_cn = ' '.join(ct[:i])
            m = re.search(r'[A-Z]+', stuff_cn)
            if m == None:
                m = 1
            else:
                m = m.end()
            data_holders[-1][-1] += stuff_cn[:m - 1]
            data_holder += split_center_street(stuff_cn[m - 1:])

            #City, State, Postal_code
            city, state_postcode = ct[i].split(',')
            state, postcode = state_postcode.split()
            data_holder += [city, state, postcode]
            i += 1

            #phone + keys
            phone = ct[i]
            i += 1
            while not ct[i][0:2].isupper():
                phone += ' ' + ct[i]
                i += 1
            if phone[-1].isupper():
                key_start = re.search(r'[A-Z]{2}', phone).start()
                data_holder.append(phone[:key_start])
                data_holder.append(phone[key_start:] + ' ' + ' '.join(ct[i:]))
            else:
                key_start = len(phone)
                data_holder.append(phone)
                data_holder.append(' '.join(ct[i:]))

            #clean the keys in last line also split the other center name if present
            keys_tail = data_holder[-1]
            if not ' '.join(keys_tail).isupper():
                keys = keys_tail.split(' ')
                data_holder[-1] = ''
                i = 0
                while i < len(keys):
                    if not keys[i].isupper():
                        break
                    i += 1
                data_holder[-1] = ' '.join(keys[:i])
                morekey_cn = ' '.join(keys[i:])
                i = 2
                split = re.search(r'[A-Z 0-9]+', morekey_cn).end()
                data_holder[-1] += morekey_cn[:split-1]
                cts[j+1] = [morekey_cn[split-1:]] + cts[j+1]
            data_holders.append(data_holder)
            #print(data_holders, '\n')
        j += 1
    for dh in data_holders:
        writer.writerow(dh)

In [292]:
def run(year, start_page, end_page = 0):

    #read file
    dir_filename = '{0}/directory_{0}.pdf'.format(year)
    pdfFileObj = open(dir_filename, 'rb')
    pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
    
    #pages to scrape
    if end_page == 0:
        end_page = pdfReader.numPages - 1

    save_as_filename = '{0}/{0}.csv'.format(year)
    if os.path.isfile(save_as_filename):
        with open(save_as_filename, 'a', newline='') as file:
            writer = csv.writer(file)
    else:
        with open(save_as_filename, 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(["Year", "Page", "Center_name", "Street_address", "City", \
                             "State", "Postal_code", "Phone", "Keys"])

    for pagen in range(start_page, end_page + 1):
        page = pdfReader.getPage(pagen - 1).extractText()
        rawdata = page.split('\n')
        rawdata[:] = [l.replace('  ', ' ').replace('Clients Only', '').strip() for l in rawdata if not (l.strip() == '' or l == 'KEY')]


        if (len(rawdata) > 3):
            data = split_data(rawdata)
            
            #print(data)

            with open(save_as_filename, 'a', newline='') as file:
                writer = csv.writer(file)
                clean_data(writer, data, 2003, pagen)

In [293]:
run(2003, 423) #311

IndexError: string index out of range

In [282]:
pagen = 310
page = pdfReader.getPage(pagen - 1).extractText()
rawdata = page.split('\n')
rawdata[:] = [l.replace('  ', ' ').replace('Clients Only', '').strip() for l in rawdata if not (l.strip() == '' or l == 'KEY')]
data = split_data(rawdata)
data

[['296For Code Definitions - See',
  'KEY on Page v. and cover flap',
  'LOUISIANALake Charles ADC3505 5th Avenue',
  'Suite B',
  'Lake Charles, LA 70607',
  '(337) 475-4855',
  'SA TX OP AD CO DU CJ SF PI SS SP AH F27 F109Lake Charles Memorial HospitalThe Recovery Center'],
 ['1701 Oak Park Boulevard',
  'Lake Charles, LA 70601',
  '(337) 494-3262',
  'www.lcmh.com',
  'MH TX DT HI SF MD MC PI MI SP AHF27 F29Nepenthe House1414 Martha Street'],
 ['Lake Charles, LA 70601',
  '(337) 439-3061',
  'SA TX HH RR MN SFSaint Patrick Hospital'],
 ['Chemical Dependency Treatment Center',
  '145 Victoria Drive',
  'Lake Charles, LA 70611',
  '(337) 855-1727',
  'SA TX DT OP PH RR HI CO SF PI MI SP AH F14 F16 F27 F29 F52 F84 F85 F87 F109Stages of Recovery216 South Ryan Street'],
 ['Lake Charles, LA 70601',
  '(337) 430-0444',
  'SA TX OP AD CO SF PILEESVILLEBayne/Jones Army Community Hospital'],
 ['Army Substance Abuse Program',
  '1585 3rd Street',
  'Leesville, LA 71459',
  '(337) 531-1972',
  

In [284]:
data310 = \
[['Washington Street Hope Center Inc',
  '103 South Main Street',
  'Marksville, LA 71351',
  '(318) 253-8705',
  'SA TX RR AD SFMARRERONew Orleans Narcotic Treatment Center'],
 ['7606 Westbank Expressway',
  'Marrero, LA 70072',
  '(504) 347-1120',
  'Hotline: (800) 631-2691',
  'Methadone',
  'SA TX DT MM OP SFWestbank Addictive Disorders Clinic'],
 ['5001 Westbank Expressway',
  'Marrero, LA 70072',
  '(504) 349-8833',
  'MH-SA TX OP CO WO MN SF PI MI SSMETAIRIE'],
 ['Jefferson Addictive Disorders Clinic',
  '3101 West Napoleon Avenue',
  'Suite 2000',
  'Metairie, LA 70001',
  '(504) 838-5263',
  'Intake: (504) 846-6940',
  'Hotline: (504) 832-5123',
  'www.jphsa.org',
  'SA TX OP CO WO MN SF MD MC PI SS AHNew Freedom Inc'],
 ['3200 Edenborn Avenue',
  'Metairie, LA 70002',
  '(504) 888-8600',
  'MH-SA TX OP AD DU SF PI MIMONROEMonroe Addictive Disorders Clinic'],
 ['3200 Concordia Avenue',
  'Monroe, LA 71201',
  '(318) 362-5188',
  'www.dhh.state.la.us/oada',
  'SA TX OP PH AD WO MN DU SF PI MI SSSouthern Oaks Addiction Recovery'],
 ['4781 South Grand Street',
  'Monroe, LA 71202',
  '(318) 362-5430',
  'SA TX DT RR WO MN DU SF PI MIMORGAN CITYFairview Treatment Center'],
 ['1101 SE Boulevard',
  'Morgan City, LA 70380',
  '(985) 395-6750',
  'www.fairviewservices.com',
  'SA TX HI PW WO MN SF PI SS']]

In [288]:
pagen = 310
with open(save_as_filename, 'a', newline='') as file:
    writer = csv.writer(file)
    clean_data(writer, data310, 2003, pagen)

[[2003, 310, 'Washington Street Hope Center Inc', '103 South Main Street', 'Marksville', 'LA', '71351', '(318) 253-8705', 'SA TX RR ADSFMARRERO'], [2003, 310, 'New Orleans Narcotic Treatment Center', '7606 Westbank Expressway', 'Marrero', 'LA', '70072', '(504) 347-1120 Hotline: (800) 631-2691 Methadone', 'SA TX DT MM OPSF']] 

[[2003, 310, 'Washington Street Hope Center Inc', '103 South Main Street', 'Marksville', 'LA', '71351', '(318) 253-8705', 'SA TX RR ADSFMARRERO'], [2003, 310, 'New Orleans Narcotic Treatment Center', '7606 Westbank Expressway', 'Marrero', 'LA', '70072', '(504) 347-1120 Hotline: (800) 631-2691 Methadone', 'SA TX DT MM OPSF'], [2003, 310, 'Westbank Addictive Disorders Clinic', '5001 Westbank Expressway', 'Marrero', 'LA', '70072', '(504) 349-8833', 'MH-SA TX OP CO WO MN SF PI MI SSMETAIRIE']] 

[[2003, 310, 'Washington Street Hope Center Inc', '103 South Main Street', 'Marksville', 'LA', '71351', '(318) 253-8705', 'SA TX RR ADSFMARRERO'], [2003, 310, 'New Orleans Na